In [1]:
import pandas as pd
import numpy as np

# 2 อันล่างนี้เป็นโมเดล TF-IDF เด๋วมาอธิบายยยยย
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## ในที่นี้จะใช้ Data แค่2ไฟล์คือ anime(เพื่อหารายชื่ออนิเมะ) 
## กับ animelist(เพื่อตรวจสอบพฤติกรรมผู้ใช้คนอื่นๆ)

In [2]:
data_1 = pd.read_csv("animelist.csv")
animelist = data_1.copy()

In [3]:
data_2 = pd.read_csv("anime.csv")
anime_name = data_2.copy()

In [4]:
animelist = animelist.iloc[: , :4]
animelist

,user_id,anime_id,rating,watching_status
0,0,67,9,1
1,0,6702,7,1
2,0,242,10,1
3,0,4898,0,1
4,0,21,10,1
...,...,...,...,...
109224742,353404,507,7,2
109224743,353404,392,9,2
109224744,353404,882,6,2
109224745,353404,883,8,2


In [5]:
anime_name = anime_name.iloc[:, :9]

## Preprocessing data

In [6]:
import re
# clean ข้อมูลตัวอักขระพิเศษจากโดยการใช้ regular expression เด๋วมาอธิบาย
def cleansing(title):
    return re.sub("[^a-zA-Z0-9 ]", " ", title)

In [7]:
# ทำการ คลีนข้อมูลชื่อเผื่อมีอักขระพิเศษ ex. Steins;Gate --> SteinsGate
anime_name["clean Name"] = anime_name["Name"].apply(cleansing)

In [8]:
anime_name[anime_name["Name"] == "Steins;Gate"]

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,clean Name
5683,9253,Steins;Gate,9.11,"Thriller, Sci-Fi",Steins;Gate,STEINS;GATE,TV,24,"Apr 6, 2011 to Sep 14, 2011",Steins Gate


In [11]:
# claen ให้ genres เป็น list
anime_name["list Genres"] = anime_name["Genres"].str.split(", ")

In [12]:
anime_name.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,clean Name,list Genres
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Cowboy Bebop,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]"
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Cowboy Bebop Tengoku no Tobira,"[Action, Drama, Mystery, Sci-Fi, Space]"
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Trigun,"[Action, Sci-Fi, Adventure, Comedy, Drama, Sho..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Witch Hunter Robin,"[Action, Mystery, Police, Supernatural, Drama,..."
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Bouken Ou Beet,"[Adventure, Fantasy, Shounen, Supernatural]"


## Create Search engine

In [13]:
vectorizer = TfidfVectorizer()

# จะแปลงข้อมูลตรง clean Name เป็นตัวเลขลงใน Matrix
tfidf = vectorizer.fit_transform(anime_name["clean Name"])

In [14]:
name = "cowboy bebop"
name= cleansing(name)
query_vec = vectorizer.transform([name])
# มีการใช้ model cosine เพื่อช่วยในการหาคำที่ต้องการ
similarity = cosine_similarity(query_vec, tfidf).flatten()
indices = np.argsort(similarity)[-5:]
anime_search = anime_name.iloc[indices][::-1]

In [15]:
cosine_sim = cosine_similarity(tfidf)

In [16]:
anime_search

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,clean Name,list Genres
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Cowboy Bebop,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]"
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Cowboy Bebop Tengoku no Tobira,"[Action, Drama, Mystery, Sci-Fi, Space]"
7607,17205,Cowboy Bebop: Ein no Natsuyasumi,6.25,Adventure,Ein's Summer Vacation,アインのなつやすみ,Special,1,"Dec 21, 2012",Cowboy Bebop Ein no Natsuyasumi,[Adventure]
3454,4037,Cowboy Bebop: Yose Atsume Blues,7.44,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop Session XX,カウボーイビバップ よせあつめブルース,Special,1,"Jun 26, 1998",Cowboy Bebop Yose Atsume Blues,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]"
13580,36357,Chibikko Cowboy,Unknown,"Music, Kids",Unknown,ちびっこカウボーイ,Music,1,"Aug 25, 1966",Chibikko Cowboy,"[Music, Kids]"


In [17]:
def search(name):
    name= cleansing(name)
    query_vec = vectorizer.transform([name])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argsort(similarity)[-5:]
    anime_search = anime_name.iloc[indices][::-1]
    return anime_search

## Create Recommend System

In [18]:
animelist.head()

,user_id,anime_id,rating,watching_status
0,0,67,9,1
1,0,6702,7,1
2,0,242,10,1
3,0,4898,0,1
4,0,21,10,1


In [19]:
animeId = 1 # สมมุติให้ animeId คือ 1 ซึ่งก็คือ cowboy bebop
# หาว่าคนที่ชอบดู cowboy bebop และ ให้rating มากกว่า8 มีกี่คนโดยคืนค่าออกมาเป็น user_id
similar_anime = animelist[(animelist["anime_id"] == animeId ) & (animelist["rating"] > 8)]["user_id"].unique() # ใช้คำสั่ง unique เพื่อดูค่าที่ไม่ซ้ำ
similar_anime

array([     3,     14,     22, ..., 353380, 353391, 353395], dtype=int64)

In [20]:
#หาว่า คนที่ชอบเมะเดียวกันใน user_id ชอบเมะอื่นเรื่องอะไรบ้างอีกที่ให้ rating มากกว่า 8 โดยคืนค่าออกมาเป็น anime_id
similar_anime_rec = animelist[(animelist["user_id"].isin(similar_anime)) & (animelist["rating"] > 8)]["anime_id"]

In [21]:
# เช็คจำนวนคนที่ชอบเมะเรื่องอื่นๆตาม anime_id ว่ามีกี่คนในแต่ละเรื่อง
similar_anime_rec.value_counts()

1        53513
5114     29302
9253     25083
1535     24542
199      23435
         ...  
36592        1
39427        1
33215        1
31235        1
41340        1
Name: anime_id, Length: 16355, dtype: int64

In [22]:
# ทำเป็นอัตราส่วน จำนวนคนที่ชอบเมะcowboy bebop และเรื่องอื่นๆ / จำนวนคนทั้งหมดที่ชอบดูเมะcowboy bebop
similar_anime_rec = similar_anime_rec.value_counts() / len(similar_anime)

# เอาอัตราส่วนที่มากกว่า 10%
similar_anime_rec = similar_anime_rec[similar_anime_rec > .1]

In [23]:
similar_anime_rec

1        1.000000
5114     0.547568
9253     0.468727
1535     0.458618
199      0.437931
           ...   
7674     0.100891
15417    0.100873
37450    0.100817
268      0.100667
14741    0.100555
Name: anime_id, Length: 167, dtype: float64

In [24]:
# เอาจำนวนคนทั้งหมดที่ดูอนิเมะแนวเดียวกับ Cowboy bebop และให้ rating มากกว่า 8
# อย่าสับสนกับอันเมื่อกี้
# similar คือ กรองcowboy bebopที่ให้ rate > 8 ก่อน แล้วเรื่องอื่นอีกที่ให้ rate > 8 
# all คือ จำนวนคนทั้งหมดที่ดูทั้งหมด และให้ rate > 8 ก็คือ คนๆนั้นอาจจะดู cowboy bebop ให้rate สัก 2 แต่เรื่องอื่นที่อยู่ใน similar_anime_rec อาจจะให้ rate > 8
all_anime = animelist[(animelist["anime_id"].isin(similar_anime_rec.index)) & (animelist["rating"] > 8)]

In [26]:
all_anime.head()

,user_id,anime_id,rating,watching_status
4,0,21,10,1
16,0,121,9,2
20,0,578,10,2
48,0,2236,10,2
51,0,269,9,3


In [27]:
len(all_anime["anime_id"].value_counts())

167

In [28]:
len(all_anime["user_id"].unique())

300692

In [29]:
# จำนวนคนทั้งหมดที่ชอบ movie rec
all_anime_rec = all_anime["anime_id"].value_counts() / len(all_anime["user_id"].unique())

In [30]:
all_anime_rec.head()

1535     0.391730
5114     0.380595
9253     0.329842
16498    0.325037
1575     0.306064
Name: anime_id, dtype: float64

In [31]:
rec_percentages = pd.concat([similar_anime_rec, all_anime_rec], axis=1)
rec_percentages.columns = ["similar", "all"]

In [32]:
rec_percentages.head()

,similar,all
1,1.000000,0.177966
5114,0.547568,0.380595
9253,0.468727,0.329842
1535,0.458618,0.391730
199,0.437931,0.271447


In [33]:
# นำมาทำเป็นอัตราส่วนเพื่อตรวจสอบว่า
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [34]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)
rec_percentages

,similar,all,score
1,1.000000,0.177966,5.619046
5,0.254499,0.052020,4.892327
801,0.126904,0.038039,3.336155
467,0.169249,0.052053,3.251450
6,0.225085,0.071452,3.150170
...,...,...,...
13759,0.116084,0.125261,0.926736
20,0.129875,0.141836,0.915669
1735,0.123409,0.140692,0.877158
22319,0.114028,0.139332,0.818394


In [36]:
# เลือกอันที่อัตราส่วนสูงสุด 10 อันแรก 
rec_percentages.head(10).merge(anime_name, left_index=True, right_on="MAL_ID")[["MAL_ID", "Name", "Score", "Genres"]]

,MAL_ID,Name,Score,Genres
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space"
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space"
728,801,Koukaku Kidoutai: Stand Alone Complex 2nd GIG,8.54,"Action, Military, Sci-Fi, Mystery, Police, Mec..."
438,467,Koukaku Kidoutai: Stand Alone Complex,8.45,"Action, Military, Sci-Fi, Police, Mecha, Seinen"
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen"
24,43,Koukaku Kidoutai,8.29,"Action, Mecha, Police, Psychological, Sci-Fi, ..."
28,47,Akira,8.17,"Action, Military, Sci-Fi, Adventure, Horror, S..."
182,205,Samurai Champloo,8.5,"Action, Adventure, Comedy, Historical, Samurai..."
203,227,FLCL,8.03,"Action, Sci-Fi, Comedy, Parody, Dementia, Psyc..."
4690,6675,Redline,8.29,"Action, Sci-Fi, Cars, Sports"


In [37]:
# อันนี้แค่เขียนในรูปฟังก์ชั่น
def recommendation(animeId):
    similar_anime = animelist[(animelist["anime_id"] == animeId ) & (animelist["rating"] > 8)]["user_id"].unique()
    similar_anime_rec = animelist[(animelist["user_id"].isin(similar_anime)) & (animelist["rating"] > 8)]["anime_id"]

    similar_anime_rec = similar_anime_rec.value_counts() / len(similar_anime)
    similar_anime_rec = similar_anime_rec[similar_anime_rec > .1]

    all_anime = animelist[(animelist["anime_id"].isin(similar_anime_rec.index)) & (animelist["rating"] > 8)]
    all_anime_rec = all_anime["anime_id"].value_counts() / sum(all_anime["anime_id"].value_counts())

    rec_percentages = pd.concat([similar_anime_rec, all_anime_rec], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(anime_name, left_index=True, right_on="MAL_ID")[["MAL_ID", "Name", "Score", "Genres"]]

# เขียนให้อยู่ในรูป OOP

In [38]:
class Anime:
    def __init__(self, anime_name, anime_2, anime_3):
        self.anime = anime_name
        self.anime2 = anime_2
        self.anime3 = anime_3
        self._animeId = self._cal_malid(self.anime)
        self._animeId2 = self._cal_malid(self.anime2)
        self._animeId3 = self._cal_malid(self.anime3)
        self._gen = {}
        self.pop_gen = self._cal_gen(self.anime, self.anime2, self.anime3)
    
    def _cal_malid(self, anime):
        """ไว้หาanime id ของ anime ที่เราป้อนเข้าไป"""
        name= cleansing(anime)
        query_vec = vectorizer.transform([name])
        similarity = cosine_similarity(query_vec, tfidf).flatten()
        indices = np.argsort(similarity)[-5:]
        anime_search = anime_name.iloc[indices][::-1]
        return anime_search[["MAL_ID", "Name", "Score", "Genres", "Type"]].iloc[0]["MAL_ID"]
    
    def __calculate_recommend(self, animeid):
        """ไว้หาอนิเมะแนะนำของอนิเมะที่เราป้อนเข้าไปมา10อันดับ"""
        similar_anime = animelist[(animelist["anime_id"] == animeid ) & (animelist["rating"] > 8)]["user_id"].unique()
        similar_anime_rec = animelist[(animelist["user_id"].isin(similar_anime)) & (animelist["rating"] > 8)]["anime_id"]

        similar_anime_rec = similar_anime_rec.value_counts() / len(similar_anime)
        similar_anime_rec = similar_anime_rec[similar_anime_rec > .1]

        all_anime = animelist[(animelist["anime_id"].isin(similar_anime_rec.index)) & (animelist["rating"] > 8)]
        all_anime_rec = all_anime["anime_id"].value_counts() / sum(all_anime["anime_id"].value_counts())

        rec_percentages = pd.concat([similar_anime_rec, all_anime_rec], axis=1)
        rec_percentages.columns = ["similar", "all"]

        rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
        rec_percentages = rec_percentages.sort_values("score", ascending=False)
        results = rec_percentages.merge(anime_name, left_index=True, right_on="MAL_ID")[["MAL_ID", "Name", "Score", "Genres", "Type"]]
        return results
    
    def _search_gen(self, anime):
        """ไว้หาgenresของอนิเมะที่เราป้อนเข้าไป"""
        name= cleansing(anime)
        query_vec = vectorizer.transform([name])
        similarity = cosine_similarity(query_vec, tfidf).flatten()
        indices = np.argsort(similarity)[-5:]
        anime_search = anime_name.iloc[indices][::-1]
        return anime_search[["MAL_ID", "Name", "Score", "Genres", "Type", "list Genres"]].iloc[0]["list Genres"]
    
    
    def _count_gen(self, gen):
        """นับจำนวนgenresในรูปแบบdictionary"""
        for key in gen:
            if key in self._gen:
                self._gen[key] += 1
            else:
                self._gen[key] = 1
        
    def _cal_gen(self, ani1, ani2, ani3):
        """ไว้คำนวณหาค่าgenresสูงสุด2อันดับแรก"""
        self._count_gen(self._search_gen(ani1))
        self._count_gen(self._search_gen(ani2))
        self._count_gen(self._search_gen(ani3))
        similar_gen = dict(sorted(self._gen.items(), key=lambda item: item[1]))
        similar_gen = list(similar_gen.keys())
        similar_gen = similar_gen[-1:-3:-1]
        return similar_gen
    
    def search(self, anime):
        """ไว้เสิร์ชหาชื่ออนิเมะ"""
        name= cleansing(anime)
        query_vec = vectorizer.transform([name])
        similarity = cosine_similarity(query_vec, tfidf).flatten()
        indices = np.argsort(similarity)[-5:]
        anime_search = anime_name.iloc[indices][::-1]
        return anime_search[["MAL_ID", "Name", "Score", "Genres", "Type"]]

    def recommend(self, anime=False):
        """หาอนิเมะrecommendจากทั้ง3เรื่อง หรือจะหาจากเรื่องเดียวก็ได้"""
        if anime == False:
            rec1 = self.__calculate_recommend(self._animeId)
            rec2 = self.__calculate_recommend(self._animeId2)
            rec3 = self.__calculate_recommend(self._animeId3)
            recomm = pd.merge(rec1, rec2, how='inner')
            result = pd.merge(recomm, rec3, how='inner').iloc[1:,:]
            result = result[result["Genres"].str.contains(self.pop_gen[0]) | result["Genres"].str.contains(self.pop_gen[1])]
            return result.head(10)
        else:
            result = self.__calculate_recommend(self._cal_malid(anime))
            return result.head(10)

In [39]:
gen1 = Anime("One piece", "Jujutsu kaisen TV", "Dragon ball z")

In [41]:
gen1.pop_gen

['Shounen', 'Action']

In [40]:
gen1.recommend()

,MAL_ID,Name,Score,Genres,Type
1,22043,Fairy Tail (2014),7.73,"Action, Adventure, Comedy, Fantasy, Magic, Sho...",TV
2,223,Dragon Ball,8.0,"Adventure, Comedy, Fantasy, Martial Arts, Shou...",TV
3,6702,Fairy Tail,7.68,"Action, Adventure, Comedy, Magic, Fantasy, Sho...",TV
4,269,Bleach,7.8,"Action, Adventure, Comedy, Super Power, Supern...",TV
5,1735,Naruto: Shippuuden,8.16,"Action, Adventure, Comedy, Super Power, Martia...",TV
6,813,Dragon Ball Z,8.16,"Action, Adventure, Comedy, Fantasy, Martial Ar...",TV
7,20,Naruto,7.91,"Action, Adventure, Comedy, Super Power, Martia...",TV
8,7674,Bakuman.,8.23,"Comedy, Drama, Romance, Shounen",TV
9,14513,Magi: The Labyrinth of Magic,8.07,"Action, Adventure, Fantasy, Magic, Shounen",TV
10,18115,Magi: The Kingdom of Magic,8.28,"Action, Adventure, Magic, Fantasy, Shounen",TV
